In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from matplotlib import pyplot as plt 
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format 
pd.options.display.max_rows = None 
pd.options.display.max_columns = None 
sns.set()

In [ ]:
#wrangle fun 
def wrangle (filename , encoding , dropna_column_names= None , change_columns_names= None , date_column = None , remove_duplicated_rows_in_col = []):
    #loading data from csv
    df = pd.read_csv(filename ,encoding= encoding)
    
    #delete NaN rows in column orderid
    df.dropna(subset= dropna_column_names ,inplace=True)
    
    #Rename columns 
    df = df.rename(columns= change_columns_names )
    
    #change orderDate type to datetime 
    df[date_column] = pd.to_datetime(df[date_column])
    
    #drop duplicated rows
    df = df.drop_duplicates(subset= remove_duplicated_rows_in_col)
    
    # strip and lower case column 
    df.columns = df.columns.str.strip().str.lower()
   
    
    return df
    
   

In [ ]:
#loading data 
orders = wrangle('all_data.csv' , "latin-1" ,dropna_column_names=['OrderID'] , change_columns_names={'City.1' : 'SuppCity', 'Country.1' : 'SuppCountry' , 'Phone.1' : 'SuppPhone'} , date_column = 'OrderDate' , remove_duplicated_rows_in_col = ['OrderID'])
for cols in orders.columns:
    if 'ID' in cols or 'OrderNumber' in cols : 
        orders[cols] = orders[cols].astype(int)
        
orders = orders[['customerid','firstname','lastname','city','country','orderid','ordernumber','orderdate','totalamount']]
orders.info()

In [ ]:
#Describe TotalAmount col
orders.totalamount.describe()

In [ ]:
#Data shape
sns.histplot(data=orders , x='totalamount',kde=True)
plt.title('Distribution of total amount')
plt.xlabel('total amount')
plt.ylabel('frequency')
plt.axvline(orders.totalamount.mean() , color='red')
plt.axvline(orders.totalamount.median() , color='black')



In [ ]:
min = orders.totalamount.min() 
q1 = orders.totalamount.quantile(0.25) 
q2 = orders.totalamount.median() 
q3 = orders.totalamount.quantile(0.75) 
IQR = q3-q1 
lb = q1-(1.5*IQR)
ub = q3+(1.5*IQR)

print(f'min= {min} , q1= {q1} , q2= {q2} , q3= {q3} , IQR= {IQR} , LB= {lb : .3f} , UB= {ub}')








In [ ]:
sns.boxplot(data=orders , x='totalamount')

In [ ]:
px.box(data_frame=orders , x='totalamount')

In [ ]:
orders[orders.totalamount>15000].count() 

In [ ]:
orders = orders[orders.totalamount<15000]

In [ ]:
orders.info() 

In [ ]:
orders.city.value_counts()

In [ ]:
px.bar(data_frame=orders.city.value_counts() , title='Distribution of cities')

In [ ]:
px.bar(data_frame=orders.country.value_counts() , title='Distribution of countries')


In [ ]:
px.pie(data_frame=orders , names='country' , title='Distribution of countries')


 <h1> BI Variate analysis <h1>

In [ ]:
orders.groupby('country' , as_index=False)['totalamount'].sum().sort_values('totalamount' , ascending=False)

In [ ]:
def plots(columnname , Top10=False):
    
    #Proportion of each country's total amount = sum of total amount of each country / sum of all total amount 
    prop = orders.groupby(columnname , as_index =False)['totalamount'].sum()
    prop['proportion'] = (prop.totalamount / orders.totalamount.sum())*100
    prop = prop.sort_values('proportion' , ascending=False)
    print('Descripe of all Total amount values' , prop.totalamount.describe())
    
    if Top10==True: 
        prop = prop.head(10)
        
    fig1 = px.bar(data_frame=prop , x= columnname , y='totalamount')
    fig2 = px.pie(data_frame=prop , names= columnname , values='totalamount')
    
    fig1.show()
    fig2.show()

In [ ]:
plots("city", Top10=True)

<h3><u>Time Series analysis </u> <h3>
<ul>
  <li><p> OrderDate vs Total Amount analysis <p></li>
</ul>

In [ ]:
date_total = orders[['orderdate' , 'totalamount']]
date_total = date_total.set_index('orderdate')

In [ ]:
px.line(data_frame= date_total)

<ul>
  <li><p> OrderDate vs Total Amount analysis <p></li>
</ul>

In [ ]:
date_country = orders[['orderdate' , 'country']]
date_country = date_country.set_index('orderdate')
date_country = date_country.country.resample(rule='1M').nunique()


In [ ]:
px.line(data_frame= date_country , y='country')


<ul>
  <li><p> Customers vs Total Amount analysis <p></li>
</ul>

In [ ]:
cust_orders = {
    'fullname': orders.firstname + ' '+orders.lastname, 
    'totalamount': orders.totalamount
}

cust_orders= pd.DataFrame(cust_orders)
cust_orders.sample(5)

In [ ]:
fig = px.bar(cust_orders.groupby('fullname')['totalamount'].count().sort_values(ascending=False).head(10), text_auto=True , title='No. of orders per customer', labels={'fullname' : 'Full Name', 'value' : 'Frequency'})
#customize aspect 
fig.update_traces(marker_color='rgb(144,200,225)' , marker_line_color='rgb(0,0,0)',marker_line_width=2 , opacity=.5 , textposition='outside')

 <h1> Multi Variate analysis <h1>


In [ ]:
orders.head()

<ul>
  <li><p> OrderDate vs Country vs Total Amount analysis vs <p></li>
</ul>

In [ ]:
multivar = order